In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
# Crear columnas nuevas vacías para la información que scrapeo
df_steam_games['publisher_scraped'] = None
df_steam_games['genres_scraped'] = None
df_steam_games['app_name_scraped'] = None
# df_steam_games['title'] = None
df_steam_games['release_date_scraped'] = None
# df_steam_games['tags'] = None
# df_steam_games['reviews_url'] = None
# df_steam_games['specs'] = None
df_steam_games['price_scraped'] = None
# df_steam_games['early_access'] = None
# df_steam_games['developer'] = None

In [ ]:
#Creo una función de scrapeo pero sólo de las columnas que voy a necesitar para las funciones
def scrape_steam_game_info(url):
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to retrieve data for URL: {url}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Guardar el contenido de soup en un archivo .txt
    with open("soup_content.txt", "w", encoding="utf-8") as file:
        file.write(soup.prettify())
    
  #  print (soup)
    try:
        # Extracting information
        app_name = soup.find('div', class_='apphub_AppName').text.strip() if soup.find('div', class_='apphub_AppName') else None
        # title = app_name  # You can use app_name as title since they are the same
        grid_content = soup.find('div', class_='grid_content')
        if grid_content:
            publisher_link = grid_content.find('a')
            if publisher_link:
                publisher = publisher_link.text.strip()
        genres = [genre.text.strip() for genre in soup.find_all('a', class_='app_tag')]
        release_date = soup.find('div', class_='date').text.strip() if soup.find('div', class_='date') else None
        # tags = [tag.text.strip() for tag in soup.find_all('a', class_='app_tag')]
        # reviews_url = f"{url}/reviews/"
        # specs = soup.find('div', class_='game_area_sys_req').text.strip() if soup.find('div', class_='game_area_sys_req') else None
        price_elem = soup.find('div', class_='game_purchase_price')
        price = price_elem.text.strip() if price_elem else "Free to Play"
        # early_access = 'Early Access' in soup.find('div', class_='release_date').text if soup.find('div', class_='release_date') else False
        # developer_elem = publisher_elem[0].text.strip() if publisher_elem else None
        # developer = developer_elem

        game_info = {
            'publisher': publisher,
            'genres': genres,
            'app_name': app_name,
            # 'title': title,
            'release_date': release_date,
            # 'tags': tags,
            # 'reviews_url': reviews_url,
            # 'specs': specs,
             'price': price
            # 'early_access': early_access,
            # 'developer': developer
        }
        
        return game_info
    
    except Exception as e:
        print(f"An error occurred while scraping URL {url}: {e}")
        return None

# Supongamos que tu dataframe se llama df_steam_games y tiene una columna 'url'
def add_game_info_to_df(df):

    for index, row in df.iterrows():
        game_info = scrape_steam_game_info(row['url'])
        
        
        if game_info:
            # Asignar los valores obtenidos a las columnas correspondientes
            df.at[index, 'publisher_scraped'] = game_info['publisher']
            df.at[index, 'genres_scraped'] = game_info['genres']
            df.at[index, 'app_name_scraped'] = game_info['app_name']
        #     df.at[index, 'title'] = game_info['title']
            df.at[index, 'release_date_scraped'] = game_info['release_date']
        #     df.at[index, 'tags'] = game_info['tags']
        #     df.at[index, 'reviews_url'] = game_info['reviews_url']
        #     df.at[index, 'specs'] = game_info['specs']
            df.at[index, 'price_scraped'] = game_info['price']
        #     df.at[index, 'early_access'] = game_info['early_access']
        #     df.at[index, 'developer'] = game_info['developer']
    
    return df

# Ejecutar la función para añadir la información al dataframe

   

df_steam_games = add_game_info_to_df(df_steam_games) #iloc[0:1]